In [172]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [173]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import random
import requests
import json


### DB IGN : Routes, Points d'intérêts, Habitations

### Loading data for La Baule

In [174]:
file_ign_44 = '../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'
gdf_commune = gpd.read_file(file_ign_44, layer="commune")
gdf_la_baule = gdf_commune[gdf_commune['code_postal'] == '44500']

gdf_la_baule_gps = gdf_la_baule.to_crs(epsg=4326)
gdf_la_baule_gps = gdf_la_baule_gps.drop(columns=['date_du_recensement'])
geometry_la_baule_gps = gdf_la_baule_gps.iloc[0].geometry
center_lat, center_lon = geometry_la_baule_gps.centroid.y, geometry_la_baule_gps.centroid.x

# Routing

In [175]:
def get_random_point_in_geometry(geometry):
    """Generate a random point inside the given geometry"""
    min_x, min_y, max_x, max_y = geometry.bounds
    random_point = [random.uniform(min_x, max_x), random.uniform(min_y, max_y)]
    return random_point

In [176]:
def get_random_points_in_geometry(geometry, num_points=2):
    """Generate a specified number of random points inside the given geometry 
       and return a GeoDataFrame with their coordinates."""
    
    points = []
    coordinates = []

    # Generate the required number of points
    while len(points) < num_points:
        min_x, min_y, max_x, max_y = geometry.bounds
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if geometry.contains(random_point):
            points.append(random_point)
            coordinates.append((random_point.x, random_point.y))
    
    # Create a GeoDataFrame with the points
    gdf = gpd.GeoDataFrame(pd.DataFrame(coordinates, columns=['x', 'y']),
                           geometry=points, crs="EPSG:4326")
    
    return coordinates, gdf

In [177]:
def get_random_point_in_geometry(geometry):
    """Generate one random point inside the given geometry 
       and return a GeoDataFrame with its coordinates."""
    
    min_x, min_y, max_x, max_y = geometry.bounds
    while True:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if geometry.contains(random_point):
            # Create a GeoDataFrame with the point
            gdf = gpd.GeoDataFrame(pd.DataFrame({'x': [random_point.x], 'y': [random_point.y]}),
                                   geometry=[random_point], crs="EPSG:4326")
            return gdf

## Routing between two points

In [196]:
# Generate two random points inside the selected geometry
df_start_point = get_random_point_in_geometry(geometry_la_baule_gps)
df_end_point = get_random_point_in_geometry(geometry_la_baule_gps)

start_point = [df_start_point.iloc[0].geometry.centroid.x, df_start_point.iloc[0].geometry.centroid.y]
end_point = [df_end_point.iloc[0].geometry.centroid.x, df_end_point.iloc[0].geometry.centroid.y]

# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    gdf_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

folium.GeoJson(
    df_start_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_end_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

In [184]:
import requests
from urllib.parse import urlencode

# Base URL
base_url = "https://data.geopf.fr/navigation/itineraire"

# Parameters as a dictionary
params = {
    "resource": "bdtopo-osrm",
    "start": f"{start_point[0]},{start_point[1]}",  # New start coordinates
    "end": f"{end_point[0]},{end_point[1]}",  # New end coordinates
    "profile": "car",
    "optimization": "fastest",
    "constraints": '{"constraintType":"banned","key":"wayType","operator":"=","value":"autoroute"}',
    "getSteps": "true",
    "getBbox": "true",
    "distanceUnit": "kilometer",
    # "timeUnit": "hour",
    "timeUnit": "minute",
    "crs": "EPSG:4326",
}

# Encode parameters properly and create full URL
query_string = urlencode(params)
full_url = f"{base_url}?{query_string}"

print("Generated URL:", full_url)

# Make the request
response = requests.get(full_url, headers={"Accept": "application/json"})

# Check response
if response.status_code == 200:
    data = response.json()
    print("Response:", data)
else:
    print(f"Error: {response.status_code}, {response.text}")

data.keys()

Generated URL: https://data.geopf.fr/navigation/itineraire?resource=bdtopo-osrm&start=-2.3571076739080077%2C47.30139621507385&end=-2.3168300010671143%2C47.29434000391552&profile=car&optimization=fastest&constraints=%7B%22constraintType%22%3A%22banned%22%2C%22key%22%3A%22wayType%22%2C%22operator%22%3A%22%3D%22%2C%22value%22%3A%22autoroute%22%7D&getSteps=true&getBbox=true&distanceUnit=kilometer&timeUnit=minute&crs=EPSG%3A4326
Response: {'resource': 'bdtopo-osrm', 'resourceVersion': '2025-01-22', 'start': '-2.357113,47.301351', 'end': '-2.318469,47.295986', 'profile': 'car', 'optimization': 'fastest', 'geometry': {'coordinates': [[-2.357113, 47.301351], [-2.356823, 47.301335], [-2.356132, 47.301286], [-2.35612, 47.301257], [-2.356087, 47.301238], [-2.356052, 47.301233], [-2.356017, 47.301241], [-2.355996, 47.301257], [-2.355984, 47.301278], [-2.355084, 47.301236], [-2.354782, 47.301195], [-2.354706, 47.301166], [-2.354535, 47.301064], [-2.354412, 47.300973], [-2.354378, 47.300949], [-2.35

dict_keys(['resource', 'resourceVersion', 'start', 'end', 'profile', 'optimization', 'geometry', 'crs', 'distanceUnit', 'timeUnit', 'bbox', 'distance', 'duration', 'constraints', 'portions'])

In [185]:
# Extract important information
distance = data.get("distance")
distance_unit = data.get("distanceUnit")
duration = data.get("duration")
time_unit = data.get("timeUnit")

# Print the formatted information
print(f"Distance: {distance} {distance_unit}")
print(f"Duration: {duration} {time_unit}")

Distance: 3.5153000000000003 kilometer
Duration: 5.651666666666667 minute


In [186]:
# Convert coordinates to a LineString (longitude, latitude format)
line_geom = LineString(data['geometry']["coordinates"])

# Create a GeoDataFrame
gdf_routing_gps = gpd.GeoDataFrame([{"geometry": line_geom}], crs="EPSG:4326")  # WGS 84 CRS

# Display the GeoDataFrame
gdf_routing_gps

,geometry
0,"LINESTRING (-2.35711 47.30135, -2.35682 47.301..."


In [ ]:
# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    gdf_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Ajouter le chemin entre les points 
folium.GeoJson(
    gdf_routing_gps,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_start_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_end_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

## Reachable area from one point

In [189]:
def get_isochrone(
    point, resource="bdtopo-valhalla", cost_value=300, cost_type="time",
    profile="car", direction="departure", constraints=None,
    distance_unit="meter", time_unit="second", crs="EPSG:4326"
):
    """
    Sends a request to the GeoPF Isochrone API.

    Parameters:
        point (tuple): (longitude, latitude) of the starting location.
        resource (str): Routing engine to use (default: "bdtopo-valhalla").
        cost_value (int): Value for the cost function (e.g., time in seconds).
        cost_type (str): Cost type ("time" or "distance").
        profile (str): Transport mode ("car", "bike", "foot", etc.).
        direction (str): "departure" (outward) or "arrival" (inward).
        constraints (dict or None): Constraints on the route.
        distance_unit (str): Distance unit ("meter", "kilometer", etc.).
        time_unit (str): Time unit ("second", "minute", etc.).
        crs (str): Coordinate reference system (default is EPSG:4326).

    Returns:
        dict: The JSON response from the API.
    """

    base_url = "https://data.geopf.fr/navigation/isochrone"

    # Format constraints as a JSON string if provided
    constraints_str = json.dumps(constraints) if constraints else None

    # Define query parameters
    params = {
        "point": f"{point[0]},{point[1]}",  # Format as "longitude,latitude"
        "resource": resource,
        "costValue": cost_value,
        "costType": cost_type,
        "profile": profile,
        "direction": direction,
        "constraints": constraints_str,  # JSON-encoded constraints
        "distanceUnit": distance_unit,
        "timeUnit": time_unit,
        "crs": crs
    }

    # Send request
    response = requests.get(base_url, params=params, headers={"accept": "application/json"})

    # Check response status
    if response.status_code == 200:
        return response.json()  # Return parsed JSON response
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None  # Return None if the request fails


In [190]:
# Example: Get an isochrone for a 5-minute drive (300 seconds)
# point_coords = (2.337306, 48.849319)  # Longitude, Latitude
point_coords = start_point

# Example constraints (ban highways)
constraints_example = {
    "constraintType": "banned",
    "key": "wayType",
    "operator": "=",
    "value": "autoroute"
}

cost_value = 300
cost_type = 'time'

# Call the function
isochrone_data = get_isochrone(point_coords, cost_value=cost_value, cost_type=cost_type, constraints=constraints_example)

In [191]:
# Convert coordinates to a Shapely Polygon
polygon = Polygon(isochrone_data['geometry']["coordinates"][0])  # Extract first polygon ring

# Create a GeoDataFrame
gdf_routing_area = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")  # WGS84

In [ ]:
# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    geometry_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Ajouter la zone reachable à partir d'un point  
folium.GeoJson(
    gdf_routing_area,
    name="Reachable area",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2},
    tooltip=f"Reachable area within {cost_value} {cost_type}"
).add_to(m)

# Point de départ
folium.GeoJson(
    df_start_point,
    name="Reachable area",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2},
    tooltip=f"Starting point"
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m